In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import seml.database as db_utils
import jkutils.mpl_latex_template as mplt
import matplotlib
import os 
os.environ["PATH"] += os.pathsep + '/nfs/homedirs/schuchaj/texlive/2020/bin/x86_64-linux'

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [2]:
collection = 'cert_collective'
collection_train = 'train_collective'

jk_config = {
    'username': 'schuchaj',
    'password': 'LBGqsHEY',
    'host': 'fs.kdd.in.tum.de',
    'port': 27017,
    'db_name': 'schuchaj'
}

col = db_utils.get_collection(collection, mongodb_config=jk_config)
col_train = db_utils.get_collection(collection_train, mongodb_config=jk_config)

In [3]:
def find_items(dataset, model, max_rad, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    
    db_filter = {
        'config.restrictions.dataset' : dataset,
        'config.restrictions.model' : model,
        'config.max_rad' : max_rad,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att,
        'config.pf_minus_att': pf_minus_att,
        'config.local_budget_descriptor': None
    }
    if seed is not None:
        db_filter['config.seed'] = seed
    if col.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col.find(db_filter)
    
    ret = []
    
    
    for exp in exps:

        config = exp['config']
        if exp['result'] is None:
            continue
        cert_file_dir = exp['result']['cert_file']
        
        try:
            cert = torch.load(cert_file_dir)
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/cert_collective/{cert_file_dir.split("/")[-1]}')
            
        ret.append((exp, config, cert))
            
    return ret

In [4]:
def find_items_train(dataset, model, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    db_filter = {
        'config.dataset' : dataset,
        'config.model': model,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj
    }
    if seed is not None:
        db_filter['config.seed'] = seed

    if col_train.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col_train.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        
        run_id = config['overwrite']
        db_collection = config['db_collection']
        save_dir = config['save_dir']

        try:
            cert = torch.load(f'{save_dir}/{db_collection}_{run_id}', map_location=torch.device('cpu'))
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/train_collective/{db_collection}_{run_id}', map_location=torch.device('cpu'))

        ret.append((exp, config, cert))
            
    return ret

In [5]:
def grid_from_exp(exp, collective=True):
    cert = exp[2]
    if collective:
        return cert['collective_grid']
    else:
        idx_test = cert['idx_test']
        return ((cert['grid_base'][idx_test] > 0.5).sum(axis=0))

In [6]:
# Styling
sns.set()
pal = sns.color_palette('colorblind', 3)

In [11]:
# edges, cora

mplt.newfig(width=0.49, textwidth_pt=397.48499)

exp_collective_cora = find_items('cora_ml', 'GCN', [-1, 0, 0], 0.4, 0.00, 0, seed=0)[0]
grid_relaxed = grid_from_exp(exp_collective_cora)[:, 0, 0].astype('float')

attacked = np.array([0, 37, 66, 91, 113, 132, 151, 167, 183, 197, 210, 223, 264])
attacked = (grid_relaxed[0] - attacked) / grid_relaxed[0]
grid_relaxed /= float(grid_relaxed[0])

plt.plot(attacked, label='MILP', clip_on=False, zorder=3)
plt.plot(grid_relaxed, label='LP', clip_on=False, zorder=3)

print((attacked - grid_relaxed).max())

plt.legend()

plt.ylim(0, 1)
plt.xlim(0, 12)

plt.ylabel('Certified ratio')
plt.xlabel('Edge deletions')

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/integer/cora_adj_del', format='png', preview='png', dpi=200, tight={'pad': 0.5})

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


0.0434782608695653


In [8]:
# edges, citeseer

mplt.newfig(width=0.49, textwidth_pt=397.48499)

exp_collective_cora = find_items('citeseer', 'GCN', [-1, 0, 0], 0.4, 0.00, 0, seed=0)[0]
grid_relaxed = grid_from_exp(exp_collective_cora)[:, 0, 0].astype('float')

attacked = np.array([0, 12, 22, 32, 41, 50, 58, 66, 74, 81, 88, 94, 106])
attacked = (grid_relaxed[0] - attacked) / grid_relaxed[0]
grid_relaxed /= float(grid_relaxed[0])

plt.plot(attacked, label='MILP', clip_on=False, zorder=3)
plt.plot(grid_relaxed, label='LP', clip_on=False, zorder=3)

plt.legend()

plt.ylim(0, 1)
plt.xlim(0, 12)

plt.ylabel('Certified ratio')
plt.xlabel('Edge deletions')

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/integer/citeseer_adj_del', format='png', preview='png', dpi=200, tight={'pad': 0.5})

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [15]:
# attributes, cora_ml

mplt.newfig(width=0.49, textwidth_pt=397.48499)

exp_collective_cora = find_items('cora_ml', 'GCN', [0, 0, -1], 0, 0.002, 0.6, seed=0)[0]
grid_relaxed = grid_from_exp(exp_collective_cora)[0, 0, :].astype('float')

attacked = np.array([0, 51, 90, 119, 150, 177, 198, 220, 244, 265])
attacked = (grid_relaxed[0] - attacked) / grid_relaxed[0]
grid_relaxed /= float(grid_relaxed[0])

plt.plot(attacked[:10], label='MILP', clip_on=False, zorder=3)
plt.plot(grid_relaxed[:10], label='LP', clip_on=False, zorder=3)

plt.legend()

plt.ylim(0, 1)
plt.xlim(0, 9)

print((attacked-grid_relaxed[:10]).max())


plt.ylabel('Certified ratio')
plt.xlabel('Attribute deletions')


mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/integer/cora_attr_del', format='png', preview='png', dpi=200, tight={'pad': 0.5})

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


0.03952569169960485


In [12]:
# attributes, cora_ml

mplt.newfig(width=0.49, textwidth_pt=397.48499)

exp_collective_cora = find_items('citeseer', 'GCN', [0, 0, -1], 0, 0.002, 0.6, seed=0)[0]
grid_relaxed = grid_from_exp(exp_collective_cora)[0, 0, :].astype('float')

attacked = np.array([0, 28, 48, 63, 76, 88, 100, 111, 121, 131, 140, 149, 158, 167, 176, 184, 192, ])
attacked = (grid_relaxed[0] - attacked) / grid_relaxed[0]
grid_relaxed /= float(grid_relaxed[0])

plt.plot(attacked[:10], label='MILP', clip_on=False, zorder=3)
plt.plot(grid_relaxed[:10], label='LP', clip_on=False, zorder=3)

plt.legend()


plt.ylim(0, 1)
plt.xlim(0, 9)

plt.ylabel('Certified ratio')
plt.xlabel('Attribute deletions')

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/integer/citeseer_attr_del', format='png', preview='png', dpi=200, tight={'pad': 0.5})

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


ValueError: operands could not be broadcast together with shapes (17,) (22,) 